# 🏭 AI Supply Assistant - Prezentacja Akademicka

## Inteligentny Asystent Zakupowy dla Systemów ERP

---

**Autor:** Projekt Dyplomowy  
**Data:** Styczeń 2026  
**Wersja:** 1.4.0

---

### Spis treści:
1. Wprowadzenie i cel projektu
2. Architektura systemu
3. Technologie i narzędzia
4. Demonstracja funkcjonalności
5. Analiza danych - przykłady
6. System AI i modele LLM
7. Bezpieczeństwo
8. Podsumowanie

## 1. 📋 Wprowadzenie i Cel Projektu

### Problem biznesowy

Działy zakupów w firmach produkcyjnych borykają się z następującymi wyzwaniami:

- **Brak predykcji popytu** - zamówienia realizowane "na oko"
- **Nieefektywne zarządzanie zapasami** - nadmiarowe stany lub braki
- **Czasochłonna analiza** - ręczne przeglądanie raportów ERP
- **Brak wykrywania anomalii** - nagłe wzrosty/spadki zużycia niezauważone

### Rozwiązanie

**AI Supply Assistant** - aplikacja integrująca się z systemem ERP (Comarch Optima / CTI Produkcja), oferująca:

| Funkcja | Wartość biznesowa |
|---------|-------------------|
| Analiza trendów | Wizualizacja historycznego zużycia surowców |
| Predykcja ML | Prognozowanie popytu na podstawie danych historycznych |
| AI Assistant | Inteligentne rekomendacje zakupowe |
| BOM Analysis | Analiza zapotrzebowania dla wyrobów gotowych |

## 2. 🏗️ Architektura Systemu

```
┌─────────────────────────────────────────────────────────────────┐
│                     WARSTWA PREZENTACJI                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │              Streamlit Web Application                   │   │
│  │  ┌─────────┬─────────┬─────────┬─────────┬─────────┐   │   │
│  │  │ Analiza │Predykcja│AI Assist│ Admin   │ Login   │   │   │
│  │  └─────────┴─────────┴─────────┴─────────┴─────────┘   │   │
│  └─────────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────────┘
                              │
┌─────────────────────────────────────────────────────────────────┐
│                     WARSTWA LOGIKI BIZNESOWEJ                   │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │ Forecasting  │  │  AI Engine   │  │   Security   │          │
│  │ (ML Models)  │  │ (LLM/Gemini) │  │  (Auth/RBAC) │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
└─────────────────────────────────────────────────────────────────┘
                              │
┌─────────────────────────────────────────────────────────────────┐
│                     WARSTWA DANYCH                              │
│  ┌──────────────────────────────────────────────────────────┐  │
│  │              MS SQL Server (Comarch Optima)               │  │
│  │    CDN.Towary | CtiZlecenieNag | CtiTechnolNag | etc.    │  │
│  └──────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────┘
```

## 3. 🛠️ Technologie i Narzędzia

### Backend / Core

| Technologia | Zastosowanie | Wersja |
|-------------|--------------|--------|
| **Python** | Główny język programowania | 3.9+ |
| **Streamlit** | Framework GUI (web) | 1.30+ |
| **SQLAlchemy** | ORM / Connection pooling | 2.0+ |
| **Pandas** | Przetwarzanie danych | 2.0+ |

### Machine Learning

| Technologia | Zastosowanie |
|-------------|-------------|
| **scikit-learn** | Random Forest, Gradient Boosting |
| **statsmodels** | Exponential Smoothing |
| **llama-cpp-python** | Lokalne modele LLM (GGUF) |

### AI / LLM

| Model | Typ | Rozmiar | Kontekst |
|-------|-----|---------|----------|
| Qwen2.5-7B-Instruct | Lokalny (GGUF) | 3.8 GB | 32k tokenów |
| Qwen2.5-3B-Instruct | Lokalny (GGUF) | 2.0 GB | 32k tokenów |
| Google Gemini | Cloud API | - | 128k tokenów |

### Bezpieczeństwo

| Technologia | Zastosowanie |
|-------------|-------------|
| **bcrypt** | Hashowanie haseł użytkowników |
| **python-dotenv** | Zarządzanie sekretami (.env) |

## 4. 🎯 Demonstracja Funkcjonalności

### 4.1 System Logowania i Uprawnień

System implementuje **Role-Based Access Control (RBAC)**:

| Rola | Uprawnienia |
|------|-------------|
| **Administrator** | Pełny dostęp: zmiana bazy, kreator połączenia, zarządzanie użytkownikami |
| **Zakupowiec** | Dostęp do analizy, predykcji i AI - bez możliwości zmiany konfiguracji |

**Bezpieczeństwo haseł:**
- Algorytm: bcrypt (12 rund)
- Salt: unikalny dla każdego hasła
- Przechowywanie: JSON (bez plaintekst)

In [ ]:
# Przykład hashowania hasła w systemie
import bcrypt

# Tworzenie hash'a
password = "admin123"
salt = bcrypt.gensalt(rounds=12)
hashed = bcrypt.hashpw(password.encode('utf-8'), salt)

print(f"Hasło: {password}")
print(f"Hash: {hashed.decode('utf-8')}")
print(f"\nDługość hash'a: {len(hashed)} bajtów")

# Weryfikacja
is_valid = bcrypt.checkpw(password.encode('utf-8'), hashed)
print(f"Weryfikacja poprawna: {is_valid}")

### 4.2 Kreator Połączenia z Bazą Danych

Aplikacja automatycznie wykrywa lokalne instancje SQL Server z rejestru Windows:

```python
# Fragment kodu odkrywania serwerów
import winreg

def discover_sql_servers():
    servers = []
    registry_paths = [
        r"SOFTWARE\Microsoft\Microsoft SQL Server\Instance Names\SQL",
        r"SOFTWARE\WOW6432Node\Microsoft\Microsoft SQL Server\Instance Names\SQL"
    ]
    
    for path in registry_paths:
        try:
            with winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, path) as key:
                # Odczytaj instancje...
                pass
        except FileNotFoundError:
            continue
    
    return servers
```

**Przepływ konfiguracji:**
1. 🖥️ Wykrycie serwera
2. 🔐 Wybór uwierzytelniania (SQL/Windows)
3. 🗄️ Lista baz danych
4. ✅ Test i zapis do `.env`

## 5. 📊 Analiza Danych - Przykłady

### 5.1 Połączenie z bazą danych

In [ ]:
# Import modułów projektu
import sys
sys.path.insert(0, '..')  # Dodanie katalogu nadrzędnego do ścieżki

from src.db_connector import DatabaseConnector
import pandas as pd

# Inicjalizacja połączenia
try:
    db = DatabaseConnector(database_name='cdn_test')
    print(f"✅ Połączono z bazą: {db.database_name}")
    print(f"   Test połączenia: {db.test_connection()}")
except Exception as e:
    print(f"❌ Błąd połączenia: {e}")

### 5.2 Pobieranie danych historycznych

In [ ]:
# Pobierz dane historyczne zużycia surowców
try:
    df_hist = db.get_historical_data()
    print(f"📊 Pobrano {len(df_hist)} rekordów")
    print(f"   Zakres dat: {df_hist['Year'].min()}-W{df_hist['Week'].min()} do {df_hist['Year'].max()}-W{df_hist['Week'].max()}")
    print(f"   Unikalne produkty: {df_hist['TowarId'].nunique()}")
    display(df_hist.head(10))
except Exception as e:
    print(f"❌ Błąd: {e}")

### 5.3 Analiza stanów magazynowych

In [ ]:
# Pobierz aktualne stany magazynowe
try:
    df_stock = db.get_current_stock()
    print(f"📦 Produkty ze stanem: {len(df_stock)}")
    print(f"   Suma stanów: {df_stock['StockLevel'].sum():,.0f}")
    
    # Top 10 najczęściej używanych surowców
    print("\n🔝 Top 10 najczęściej używanych surowców:")
    display(df_stock.head(10)[['Code', 'Name', 'StockLevel', 'UsageCount']])
except Exception as e:
    print(f"❌ Błąd: {e}")

### 5.4 Filtrowanie po magazynach

In [ ]:
# Pobierz listę magazynów
try:
    df_warehouses = db.get_warehouses(only_with_stock=True)
    print("🏭 Dostępne magazyny:")
    display(df_warehouses)
    
    # Stany dla wybranego magazynu
    if not df_warehouses.empty:
        first_mag_id = df_warehouses.iloc[0]['MagId']
        df_stock_filtered = db.get_current_stock(warehouse_ids=[first_mag_id])
        print(f"\n📊 Stany dla magazynu {df_warehouses.iloc[0]['Symbol']}: {len(df_stock_filtered)} produktów")
except Exception as e:
    print(f"❌ Błąd: {e}")

### 5.5 Analiza BOM (Bill of Materials)

In [ ]:
# Pobierz wyroby z technologią (BOM)
try:
    df_tech = db.get_products_with_technology()
    print(f"🔧 Wyroby z zdefiniowaną technologią: {len(df_tech)}")
    display(df_tech.head(5))
    
    # BOM dla pierwszego wyrobu
    if not df_tech.empty:
        prod_id = df_tech.iloc[0]['FinalProductId']
        prod_name = df_tech.iloc[0]['Name']
        
        df_bom = db.get_bom_with_stock(prod_id)
        print(f"\n📋 BOM dla: {prod_name}")
        display(df_bom)
except Exception as e:
    print(f"❌ Błąd: {e}")

## 6. 🤖 System AI i Modele LLM

### 6.1 Architektura AI Engine

System oferuje trzy opcje silnika AI:

| Silnik | Prywatność | Koszt | Jakość |
|--------|------------|-------|--------|
| **Local LLM (Qwen2.5)** | 🔒 100% lokalne | Darmowy | ⭐⭐⭐⭐ |
| **Ollama** | 🔒 Lokalne | Darmowy | ⭐⭐⭐ |
| **Google Gemini** | ☁️ Cloud | Płatny | ⭐⭐⭐⭐⭐ |

### 6.2 Przykład promptu AI

In [ ]:
# Przykładowy prompt generowany przez system
product_name = "Stal nierdzewna 304"
current_stock = 1500.0
last_4_weeks = [120, 135, 142, 150]
avg_consumption = sum(last_4_weeks) / len(last_4_weeks)
warehouse_context = "(wybrane magazyny: 2)"

prompt = f"""
Jesteś ekspertem ds. łańcucha dostaw. Przeanalizuj sytuację dla surowca: {product_name}.

DANE:
- Obecny stan magazynowy {warehouse_context}: {current_stock:.2f}
- Ostatnie 4 tygodnie zużycia: {last_4_weeks}
- Średnie zużycie tygodniowe: {avg_consumption:.2f}

PYTANIA:
1. Czy trend zużycia jest rosnący czy malejący?
2. Na ile tygodni wystarczy obecny zapas (Coverage)?
3. Czy sugerujesz zwiększenie zapasów?
4. Jakie mogą być przyczyny anomalii (jeśli występują)?

Odpowiedz krótko i konkretnie w języku polskim.
"""

print("📝 Wygenerowany prompt dla AI:")
print("-" * 50)
print(prompt)

### 6.3 Test lokalnego modelu LLM

In [ ]:
# Test Local LLM (wymaga zainstalowanego modelu)
try:
    from src.ai_engine.local_llm import LocalLLMEngine, check_local_llm_available
    
    available, status = check_local_llm_available()
    print(f"🤖 Local LLM dostępny: {available}")
    print(f"   Status: {status}")
    
    if available:
        llm = LocalLLMEngine()
        
        # Prosty test
        test_prompt = "Wymień 3 powody, dla których automatyzacja zakupów jest ważna w produkcji."
        print(f"\n💭 Prompt testowy: {test_prompt}")
        print("\n🔄 Generowanie odpowiedzi...")
        
        import time
        start = time.time()
        response = llm.generate_explanation(test_prompt)
        duration = time.time() - start
        
        print(f"\n✅ Odpowiedź ({duration:.1f}s):")
        print(response)
except ImportError as e:
    print(f"⚠️ Moduł niedostępny: {e}")
except Exception as e:
    print(f"❌ Błąd: {e}")

## 7. 🔐 Bezpieczeństwo

### 7.1 Zabezpieczenia aplikacji

| Obszar | Mechanizm |
|--------|----------|
| **Uwierzytelnianie** | bcrypt (12 rund), session-based |
| **Autoryzacja** | RBAC (Admin/Zakupowiec) |
| **Baza danych** | Parametryzowane zapytania SQL |
| **Dane wrażliwe** | Anonimizacja NIP/PESEL/email przed wysyłką do Cloud |
| **Sekrety** | Zmienne środowiskowe (.env) |

### 7.2 Przykład parametryzowanego zapytania

In [ ]:
# Bezpieczne zapytanie z parametrami (zapobiega SQL Injection)
from sqlalchemy import text

# ❌ NIEBEZPIECZNE (podatne na SQL Injection)
product_id = "1; DROP TABLE users; --"  # Złośliwy input
unsafe_query = f"SELECT * FROM Towary WHERE TwrId = {product_id}"
print(f"❌ Niebezpieczne zapytanie:")
print(f"   {unsafe_query}")

# ✅ BEZPIECZNE (z parametrami)
safe_query = text("SELECT * FROM Towary WHERE TwrId = :product_id")
params = {"product_id": product_id}
print(f"\n✅ Bezpieczne zapytanie:")
print(f"   Query: {safe_query}")
print(f"   Params: {params}")
print(f"   -> Złośliwy input traktowany jako string, nie jako kod SQL")

## 8. 📈 Podsumowanie

### Kluczowe osiągnięcia projektu

1. **Integracja z ERP** - bezpośredni dostęp do danych produkcyjnych Comarch Optima
2. **Predykcja ML** - 3 modele (RF, GB, ES) do prognozowania popytu
3. **AI Assistant** - lokalne LLM (prywatność) lub Cloud (jakość)
4. **System bezpieczeństwa** - RBAC, bcrypt, parametryzowane SQL
5. **Intuicyjny GUI** - Streamlit z wieloma widokami

### Metryki projektu

| Metryka | Wartość |
|---------|--------|
| Linie kodu | ~5,000 |
| Moduły Python | 15+ |
| Widoki GUI | 5 (Analiza, Predykcja, AI, Admin, Login) |
| Modele AI | 3 (Local LLM, Ollama, Gemini) |
| Wersja | 1.4.0 |

### Potencjalne rozszerzenia

- 📱 Aplikacja mobilna (React Native)
- 🔔 System alertów i powiadomień
- 📊 Dashboard BI z raportami
- 🤝 Integracja z dostawcami (EDI)

---

**Dziękuję za uwagę!**

*Projekt dostępny na GitHub: [ai-supply-assistant](https://github.com/)*